# Hyperparameter Tuning in ANN (Keras + Keras Tuner)



In real-world applications, we do **hyperparameter tuning on a subset of training data** to save computation, then **retrain the best model on full data** for production use.

### Workflow:
1. Use **Keras Tuner** to find the best configuration:
   - Number of hidden layers
   - Neurons per layer
   - Learning rate
2. Tune using a **10,000-sample subset**
3. **Retrain** best model on **full training data**
4. Evaluate on test data


In [ ]:
!pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 4.0 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import keras_tuner as kt

# Load Fashion MNIST
(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Flatten for ANN
x_train = x_train.reshape(-1, 784)
x_test = x_test.reshape(-1, 784)

# Subset for tuning (10k samples)
x_tune = x_train[:10000]
y_tune = y_train[:10000]

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
def build_model(hp):
    model = keras.Sequential()
    model.add(keras.Input(shape=(784,)))

    for i in range(hp.Int("num_layers", 1, 3)):
        model.add(
            layers.Dense(
                units=hp.Int(f"units_{i}", min_value=32, max_value=256, step=32),
                activation="relu"
            )
        )

    model.add(layers.Dense(10, activation="softmax"))

    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice("lr", [1e-2, 1e-3, 1e-4])
        ),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model


In [ ]:
tuner = kt.RandomSearch(
    build_model,
    objective="val_accuracy",
    max_trials=5,
    executions_per_trial=1,
    directory="realistic_ann_tuner",
    project_name="fashion_ann_subset"
)

In [ ]:
tuner.search(x_tune, y_tune, epochs=10, validation_split=0.2)

Trial 5 Complete [00h 00m 14s]
val_accuracy: 0.8335000276565552

Best val_accuracy So Far: 0.8585000038146973
Total elapsed time: 00h 02m 03s


## Best Hyperparameters Found

In [ ]:
best_hps = tuner.get_best_hyperparameters(1)[0]

print("Best Hyperparameters:")
print(f"Number of Layers: {best_hps.get('num_layers')}")
for i in range(best_hps.get('num_layers')):
    print(f"Units in Layer {i+1}: {best_hps.get(f'units_{i}')}")
print(f"Learning Rate: {best_hps.get('lr')}")

Best Hyperparameters:
Number of Layers: 1
Units in Layer 1: 160
Learning Rate: 0.001


## Final Model Training on Full Dataset
Now, we'll build a model using the best hyperparameters and train it from scratch on the **entire training set (60,000 samples)**.

In [ ]:
# Rebuild and retrain
final_model = tuner.hypermodel.build(best_hps)
final_model.fit(x_train, y_train, epochs=10, validation_split=0.1)

Epoch 1/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - accuracy: 0.7754 - loss: 0.6393 - val_accuracy: 0.8603 - val_loss: 0.3912
Epoch 2/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.8622 - loss: 0.3837 - val_accuracy: 0.8430 - val_loss: 0.4264
Epoch 3/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.8739 - loss: 0.3475 - val_accuracy: 0.8713 - val_loss: 0.3436
Epoch 4/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.8837 - loss: 0.3199 - val_accuracy: 0.8768 - val_loss: 0.3509
Epoch 5/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.8917 - loss: 0.2966 - val_accuracy: 0.8725 - val_loss: 0.3535
Epoch 6/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 11s 5ms/step - accuracy: 0.8961 - loss: 0.2807 - val_accuracy: 0.8810 - val_loss: 0.3294
Epoch 7/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.9023 - loss: 0.2672 - val_accuracy: 0.8893 - val_loss: 0.3169
Epoch 8/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.9047 - loss: 0.2550

In [ ]:
test_loss, test_acc = final_model.evaluate(x_test, y_test)
print(f"Test Accuracy: {test_acc:.4f}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8796 - loss: 0.3397
Test Accuracy: 0.8817


# CNN Hyperparameter Tuning on CIFAR-10 with Keras Tuner



We'll train a Convolutional Neural Network on the CIFAR-10 dataset using the following real-world workflow:

### Steps:
1. Tune the model using a **subset (10k images)** for faster exploration.
2. Use **Keras Tuner (RandomSearch)** to explore:
   - Number of Conv layers
   - Filters per layer
   - Dense layer size
   - Learning rate
3. Retrain the **best model from scratch** on **full training data (50k samples)**.
4. Evaluate on **test set (10k images)**.


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import keras_tuner as kt

# Load CIFAR-10
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Subset for tuning (10k samples)
x_tune = x_train[:10000]
y_tune = y_train[:10000]

In [ ]:
def build_model(hp):
    inputs = keras.Input(shape=(32, 32, 3))
    x = inputs

    # Convolutional blocks
    for i in range(hp.Int("num_conv_blocks", 1, 3)):
        x = layers.Conv2D(
            filters=hp.Int(f"filters_{i}", 32, 128, step=32),
            kernel_size=3,
            padding="same",
            activation="relu"
        )(x)
        x = layers.MaxPooling2D()(x)

    x = layers.Flatten()(x)

    # Dense layer
    x = layers.Dense(
        units=hp.Int("dense_units", 64, 256, step=64),
        activation="relu"
    )(x)

    outputs = layers.Dense(10, activation="softmax")(x)

    model = keras.Model(inputs=inputs, outputs=outputs)

    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice("lr", [1e-2, 1e-3, 1e-4])
        ),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )

    return model

In [ ]:
tuner = kt.RandomSearch(
    build_model,
    objective="val_accuracy",
    max_trials=5,
    executions_per_trial=1,
    directory="cnn_tuner_results",
    project_name="cifar10_cnn2"
)

In [ ]:
tuner.search(x_tune, y_tune, epochs=10, validation_split=0.2)

Trial 10 Complete [00h 00m 23s]
val_accuracy: 0.5270000100135803

Best val_accuracy So Far: 0.5350000262260437
Total elapsed time: 00h 03m 54s


## Best Hyperparameters from Tuner

In [ ]:
best_hps = tuner.get_best_hyperparameters(1)[0]

print("Best Hyperparameters:")
print(f"Conv Blocks: {best_hps.get('num_conv_blocks')}")
for i in range(best_hps.get('num_conv_blocks')):
    print(f"Filters in Block {i+1}: {best_hps.get(f'filters_{i}')}")
print(f"Dense Units: {best_hps.get('dense_units')}")
print(f"Learning Rate: {best_hps.get('lr')}")

Best Hyperparameters:
Conv Blocks: 1
Filters in Block 1: 128
Dense Units: 192
Learning Rate: 0.0001


## Final Model Training on Full CIFAR-10 Training Set
Now we build the best model again and train it from scratch using the **entire training set (50,000 samples)**.

In [ ]:
final_model = tuner.hypermodel.build(best_hps)

final_model.fit(x_train, y_train, epochs=10, validation_split=0.1)

Epoch 1/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 12s 7ms/step - accuracy: 0.3605 - loss: 1.8053 - val_accuracy: 0.5112 - val_loss: 1.4078
Epoch 2/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.5306 - loss: 1.3471 - val_accuracy: 0.5706 - val_loss: 1.2571
Epoch 3/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - accuracy: 0.5772 - loss: 1.2216 - val_accuracy: 0.5876 - val_loss: 1.1982
Epoch 4/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.6086 - loss: 1.1267 - val_accuracy: 0.6048 - val_loss: 1.1418
Epoch 5/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 8s 6ms/step - accuracy: 0.6302 - loss: 1.0709 - val_accuracy: 0.6194 - val_loss: 1.1085
Epoch 6/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - accuracy: 0.6498 - loss: 1.0192 - val_accuracy: 0.6274 - val_loss: 1.0677
Epoch 7/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.6614 - loss: 0.9860 - val_accuracy: 0.6454 - val_loss: 1.0456
Epoch 8/10
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.6829 - loss: 0.936

In [ ]:
test_loss, test_acc = final_model.evaluate(x_test, y_test)
print(f"Test Accuracy: {test_acc:.4f}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6501 - loss: 0.9936
Test Accuracy: 0.6451
